In [3]:
import torch
torch.cuda.is_available()

True

In [1]:
import tensorflow as tf
#pipimport tensorflow_hub as hub
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Resizing


class ViTFeatureExtractorModel(Model):
    def __init__(self, num_outputs):
        super(ViTFeatureExtractorModel, self).__init__()
        # Load the pre-trained ViT model from TensorFlow Hub
        """self.vit_model = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_b16_fe/1", 
                                        trainable=False, input_shape=(224, 224, 3))"""
        self.mobilenet = MobileNetV2(include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='imagenet')
        self.mobilenet.trainable = False  # Freeze MobileNetV2
        self.resizing = Resizing(224, 224)
        # Define the model for the RGB input
        self.rgb_input = Input(shape=(224, 224, 3), name='rgb_input')
        # Define the model for the RGB depth input
        self.depth_input = Input(shape=(224, 224, 3), name='depth_input')
        
        # You can add additional layers here if needed
        # For example, Dense layers for further processing the concatenated features
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(num_outputs, activation='linear')  # Adjust based on your output needs

    def call(self, inputs):
        rgb_input, depth_input = inputs[0], inputs[1]
        #rgb_input, depth_input = inputs[0], inputs[1]
        # Resize inputs
        rgb_input = self.resizing(rgb_input)
        depth_input = self.resizing(depth_input)
        # Extract features using the ViT model
        rgb_features = self.mobilenet(rgb_input)
        depth_features = self.mobilenet(depth_input)
        
        # Concatenate features from both inputs
        concatenated_features = concatenate([rgb_features, depth_features])
        
        # Further processing and output
        x = self.dense1(concatenated_features)
        return self.dense2(x)

# Instantiate the model
num_outputs = 5  # For example, predicting 5 different properties
model = ViTFeatureExtractorModel(num_outputs=num_outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Or another appropriate loss function
              metrics=['accuracy'])


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2024-03-21 13:39:04.097661: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 13:39:04.376247: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 13:39:05.533468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-21 13:39:07.760721: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

[]

In [4]:

# To summarize the model, you need to build it first by calling it on some sample data
sample_rgb_input = tf.zeros([1, 224, 224, 3])
sample_depth_input = tf.zeros([1, 224, 224, 3])
model([sample_rgb_input, sample_depth_input])  # This builds the model
model.summary()

Model: "vi_t_feature_extractor_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │       327,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,437 (9.87 MB)

 Trainable params: 328,453 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
import pandas as pd
import numpy as np
data = np.array(pd.read_pickle('df.pkl'))
depths = pd.read_pickle('depths.pkl')
images = pd.read_pickle('images.pkl')

In [6]:
X = [depths, images]

In [7]:
del depths, images

In [8]:
data = data[:,1:].astype('float32')

In [9]:
history = model.fit(X, data, epochs=10, batch_size=32, validation_split=0.2, verbose = 1)  # Example training loop

Epoch 1/10
80/82 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step - accuracy: 0.4866 - loss: 24917.5879

In [10]:
g,h = np.array([1,2])